<a href="https://colab.research.google.com/github/alivarastepour/diabetes_prediction/blob/master/diabetes_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose of this notebook
This notebook aims to build a model that determines whether a person is prone to diabetes or not. Additionally, it seeks to identify a subset of features (risk factors) that can accurately predict the risk of diabetes. The weights of the optimal solution will be utilized in another project, where they will be applied to users' inputs in real time.

## Dataset
This notebook makes use of a subset of a larger dataset which aimed to collect uniform, state-specific data on preventive health practices and risk behaviors that are associated with chronic diseases, injuries, and preventable infectious diseases in the adult population. The subset used in this notebook can be accessed [here](https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset?select=diabetes_binary_5050split_health_indicators_BRFSS2015.csv).

In [68]:
import pandas as pd
import numpy as np
from google.colab import drive

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LeakyReLU,Dropout
from keras.optimizers import Adagrad, RMSprop, Adam
from keras.regularizers import l1, l2
from keras.initializers import he_normal
from keras.activations import selu
from keras.metrics import Precision, Recall
from sklearn.decomposition import PCA

In [2]:
drive.mount('/drive')
DATASET_ADDRESS = '/drive/MyDrive/diabetes_info.csv'
raw_dataset = pd.read_csv(DATASET_ADDRESS)

Mounted at /drive


In [3]:
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70692 entries, 0 to 70691
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Diabetes_binary       70692 non-null  float64
 1   HighBP                70692 non-null  float64
 2   HighChol              70692 non-null  float64
 3   CholCheck             70692 non-null  float64
 4   BMI                   70692 non-null  float64
 5   Smoker                70692 non-null  float64
 6   Stroke                70692 non-null  float64
 7   HeartDiseaseorAttack  70692 non-null  float64
 8   PhysActivity          70692 non-null  float64
 9   Fruits                70692 non-null  float64
 10  Veggies               70692 non-null  float64
 11  HvyAlcoholConsump     70692 non-null  float64
 12  AnyHealthcare         70692 non-null  float64
 13  NoDocbcCost           70692 non-null  float64
 14  GenHlth               70692 non-null  float64
 15  MentHlth           

In [4]:
y = raw_dataset["Diabetes_binary"]
x = raw_dataset.drop(columns=["Diabetes_binary"])

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=9)

# Model selection
While our data may appear relatively clean, this does not guarantee optimal performance. Therefore, we must leverage a range of machine learning models to assess their effectiveness and identify potential modifications to the original data that can enhance the performance of our models.

## First model: Gradient boost classifier
Boosting algorithms have been widely recognized as effective choices for handling tabular data. Among them, gradient boosting stands out as a prominent technique that leverages decision trees to create a powerful ensemble model. Nonetheless, to ensure its optimal performance, careful consideration should be given to hyperparameter tuning.

In [6]:
def get_data(dataset):
  y = dataset["Diabetes_binary"]
  x = dataset.drop(columns=["Diabetes_binary"])
  return train_test_split(x, y, test_size=0.20, random_state=9)

In [7]:
def gradient_boost_classifier_model(dataset, learning_rate=0.05, n_estimators=150, subsample=0.8):
  x_train, x_test, y_train, y_test = get_data(dataset)
  reg = GradientBoostingClassifier(random_state=90,
                                loss='deviance',
                                learning_rate=learning_rate,
                                n_estimators=n_estimators,
                                subsample=subsample,
                                criterion='friedman_mse',
                                verbose=2,
                                )
  reg.fit(x_train, y_train)
  y_pred = reg.predict(x_test)
  report = classification_report(y_test, y_pred)
  print(report)

In [8]:
gradient_boost_classifier_model(raw_dataset)

      Iter       Train Loss      OOB Improve   Remaining Time 


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


         1           1.3619           0.0245           10.46s
         2           1.3400           0.0224            9.90s
         3           1.3200           0.0203            9.96s
         4           1.3008           0.0178            9.80s
         5           1.2841           0.0163            9.61s
         6           1.2685           0.0151            9.39s
         7           1.2545           0.0136            9.39s
         8           1.2417           0.0127            9.24s
         9           1.2303           0.0117            9.12s
        10           1.2191           0.0112            8.98s
        11           1.2090           0.0101            8.87s
        12           1.1982           0.0100            8.76s
        13           1.1890           0.0094            8.71s
        14           1.1793           0.0088            8.61s
        15           1.1707           0.0078            8.56s
        16           1.1641           0.0067            8.54s
        

## The deviance loss
Deviance loss is a commonly used loss function in binary classification problems. With a glance at its formula, we can easily unserstand why:

$$
L(y, p) = \left(y \log(p) + (1 - y) \log(1 - p)\right)
$$

where y is true class and p is statistical probability.




## F-1 score
F-1 score uses precision(ratio of true possitives to true possitves and false possitives) and recall(ratio of true possitives to true possitves and false negatives) scores to prvoide a balance between them:

$$F1 = 2 \times \frac{Precision \times Recall}{Precision + Recall}$$


In [9]:
# scores = cross_val_score(reg, x_train, y_train, cv=5, scoring='f1_macro')
# print("cross validation scores(F-1) where k=5: ", scores)

In [10]:
# scores = cross_val_score(reg, x_train, y_train, cv=10, scoring='f1_macro')
# print("cross validation scores(F-1) where k=10: ", scores)

## Initial evaluataion result
As demonstrated above, whether employing Gradient Boosting with or without cross-validation, the F1 score hovers around 0.75. While this performance is acceptable, there is room for improvement.

# Second model: Logistic regression
While Logistic Regression is typically considered a more linear model compared to ensemble methods, it remains a highly prevalent choice in classification problems. It offers several distinct advantages, such as strong interpretability, feature importance insights, and the ability to not only make binary classifications but also provide class probabilities. This probabilistic aspect can prove particularly valuable in certain situations."







In [70]:
def logistic_regression_model(dataset):
  x_train, x_test, y_train, y_test = get_data(dataset)
  log_reg = LogisticRegression(random_state=32, solver='sag', multi_class='multinomial', verbose=2, max_iter=500)
  log_reg.fit(x_train, y_train)
  y_pred_log_reg = log_reg.predict(x_test)
  report_log_reg = classification_report(y_test, y_pred_log_reg)
  print(log_reg.coef_)
  print(report_log_reg)

In [67]:
logistic_regression_model(raw_dataset)

convergence after 328 epochs took 12 seconds
[[ 0.73642276  0.5859419   1.35431797  0.07424616 -0.00842411  0.17406674
   0.23032686 -0.02977515 -0.03619828 -0.08796805 -0.72938801  0.05957254
   0.01305635  0.58596997 -0.00528506 -0.00804519  0.13615712  0.26243553
   0.15226778 -0.02630124 -0.0588881 ]]
              precision    recall  f1-score   support

         0.0       0.75      0.73      0.74      7010
         1.0       0.74      0.76      0.75      7129

    accuracy                           0.74     14139
   macro avg       0.74      0.74      0.74     14139
weighted avg       0.74      0.74      0.74     14139



## Evaluation result
Logistic regression exhibited slightly lower performance compared to Gradient Boosting, indicating that additional data preprocessing may be necessary to enhance model outcomes.

### Checking for class imbalance

In [13]:
np.bincount(y)

array([35346, 35346])

## Standardizing features
In this section we standardize featuers that their domain may mislead oue models.

In [14]:
columns_to_standardize = list(x.keys())

In [15]:
scaler = StandardScaler()
standarized_features = scaler.fit_transform(raw_dataset[columns_to_standardize])
standardized_dataset = pd.DataFrame()
standardized_dataset["Diabetes_binary"] = raw_dataset["Diabetes_binary"]
standardized_dataset[columns_to_standardize] = standarized_features

In [16]:
gradient_boost_classifier_model(standardized_dataset)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3619           0.0245            9.34s
         2           1.3400           0.0224            9.54s
         3           1.3200           0.0203            9.42s
         4           1.3008           0.0178            9.69s
         5           1.2841           0.0163            9.68s
         6           1.2685           0.0151            9.56s
         7           1.2545           0.0136            9.52s
         8           1.2417           0.0127            9.55s
         9           1.2303           0.0117            9.89s
        10           1.2191           0.0112           10.40s
        11           1.2090           0.0101           10.18s
        12           1.1982           0.0100           10.01s
        13           1.1890           0.0094            9.93s
        14           1.1793           0.0088            9.77s
        15           1.1707           0.0078            9.59s
       

In [17]:
logistic_regression_model(standardized_dataset)

convergence after 35 epochs took 2 seconds
[[ 0.1826531   0.14632449  0.10666609  0.26429747 -0.0020377   0.02104763
   0.04085052 -0.00675311 -0.00880712 -0.01792627 -0.07395524  0.00629132
   0.00203814  0.32646677 -0.02152548 -0.04053807  0.02956818  0.06543934
   0.21726925 -0.01336101 -0.0640264 ]]
              precision    recall  f1-score   support

         0.0       0.75      0.73      0.74      7010
         1.0       0.74      0.76      0.75      7129

    accuracy                           0.74     14139
   macro avg       0.74      0.74      0.74     14139
weighted avg       0.74      0.74      0.74     14139



## Normalizing features
Standardization helped the convergance of our model, but didn't countribute to the evaluation metrics. Now we try with normalized data.

In [18]:
columns_to_normalize = list(x.keys())

In [19]:
min_max_scaler = MinMaxScaler()
normalized_features = min_max_scaler.fit_transform(raw_dataset[columns_to_standardize])
normalized_dataset = pd.DataFrame()
normalized_dataset["Diabetes_binary"] = raw_dataset["Diabetes_binary"]
normalized_dataset[columns_to_standardize] = normalized_features

In [20]:
gradient_boost_classifier_model(normalized_dataset)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3619           0.0245            9.12s
         2           1.3400           0.0224            9.39s
         3           1.3200           0.0203            9.43s
         4           1.3008           0.0178            9.36s
         5           1.2841           0.0163            9.22s
         6           1.2685           0.0151            9.36s
         7           1.2545           0.0136            9.23s
         8           1.2417           0.0127            9.11s
         9           1.2303           0.0117            8.96s
        10           1.2191           0.0112            8.85s
        11           1.2090           0.0101            8.82s
        12           1.1982           0.0100            8.74s
        13           1.1890           0.0094            8.62s
        14           1.1793           0.0088            8.53s
        15           1.1707           0.0078            8.47s
       

In [21]:
logistic_regression_model(normalized_dataset)

convergence after 38 epochs took 2 seconds
[[ 0.36918431  0.29307524  0.68381231  3.15904998 -0.00423009  0.08667325
   0.11512165 -0.01518912 -0.01825656 -0.04387366 -0.36555518  0.03063572
   0.00687011  1.17187081 -0.07924642 -0.1206813   0.06894083  0.13120518
   0.91025488 -0.06528242 -0.2056759 ]]
              precision    recall  f1-score   support

         0.0       0.75      0.73      0.74      7010
         1.0       0.74      0.76      0.75      7129

    accuracy                           0.74     14139
   macro avg       0.74      0.74      0.74     14139
weighted avg       0.74      0.74      0.74     14139



# What happened?
It turned out that algorithms like Logitstic regression and Gradientboost are robust to data scale due to a number of factors like their loss functions, use of decision trees and regularization factors, etc. So we have to find another way to reach our goal.

# Next model: DNN
neural networks are the master of finding complex relations between featurse. In addition to that, they can be combined with various functionalities to improve model's behavoir even further, e.g. optimizers, regularization factors, etc.

In [22]:
def dnn_model(dataset):
  x_train, x_test, y_train, y_test = get_data(dataset)
  model = Sequential()
  model.add(Dense(64, input_dim=x_train.shape[1], activation=LeakyReLU(alpha=0.1), kernel_initializer=he_normal()))
  model.add(Dropout(0.5))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(32, activation='relu', kernel_regularizer=l1(0.1)))
  model.add(Dense(1, activation='sigmoid'))
  adam = Adagrad(learning_rate=0.1)
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=[Precision(), Recall()])
  model.fit(x_train, y_train, epochs=100, verbose=2, validation_split=0.1, batch_size=100,)
  res = model.evaluate(x_test, y_test)
  print("binary cross-entropy loss : ", res[0], " precision: ", res[1], " recal: ", res[2])

In [23]:
dnn_model(standardized_dataset)

Epoch 1/100
509/509 - 5s - loss: 2.3837 - precision: 0.6401 - recall: 0.7215 - val_loss: 1.4089 - val_precision: 0.7310 - val_recall: 0.7902 - 5s/epoch - 10ms/step
Epoch 2/100
509/509 - 2s - loss: 1.2959 - precision: 0.7052 - recall: 0.8169 - val_loss: 1.1773 - val_precision: 0.6942 - val_recall: 0.8593 - 2s/epoch - 4ms/step
Epoch 3/100
509/509 - 2s - loss: 1.1039 - precision: 0.7093 - recall: 0.8221 - val_loss: 1.0201 - val_precision: 0.7001 - val_recall: 0.8586 - 2s/epoch - 3ms/step
Epoch 4/100
509/509 - 1s - loss: 1.0011 - precision: 0.7103 - recall: 0.8227 - val_loss: 0.9590 - val_precision: 0.7173 - val_recall: 0.8149 - 1s/epoch - 2ms/step
Epoch 5/100
509/509 - 1s - loss: 0.9352 - precision: 0.7112 - recall: 0.8237 - val_loss: 0.8928 - val_precision: 0.7160 - val_recall: 0.8142 - 1s/epoch - 2ms/step
Epoch 6/100
509/509 - 1s - loss: 0.8932 - precision: 0.7115 - recall: 0.8235 - val_loss: 0.8717 - val_precision: 0.7032 - val_recall: 0.8468 - 1s/epoch - 2ms/step
Epoch 7/100
509/509 -

## Result
As we saw, different models are not showing significant result improvements. So we may need to make some changes to our data

## The correlation matrix and its usage
Correlation matrix simply explains the relationship between columns of a dataset. The correlation coefficient ranges between -1 and 1. A correlation coefficient of 1 indicates a perfect positive correlation, meaning that the two variables increase or decrease together in a linear relationship. A correlation coefficient of -1 indicates a perfect negative correlation, meaning that the two variables move in opposite directions in a linear relationship. A correlation coefficient close to 0 suggests no linear relationship between the variables.

This matrix can be helpful when finding an optimal subset of features.

In [24]:
def sort_correlations(correlation_matrix):
  return sorted(correlation_matrix.items(), key=lambda x:abs(x[1]))

In [25]:
def get_correlations(dataset):
  columns = dataset.keys()
  correlation = dataset[columns].corr()
  return correlation["Diabetes_binary"]

In [26]:
correlation_map = sort_correlations(get_correlations(raw_dataset))

In [27]:
correlation_map

[('AnyHealthcare', 0.02319074853112824),
 ('NoDocbcCost', 0.040976573266643494),
 ('Sex', 0.044412858371260695),
 ('Fruits', -0.05407655628666651),
 ('Veggies', -0.07929314561269872),
 ('Smoker', 0.08599896420800192),
 ('MentHlth', 0.08702877147509416),
 ('HvyAlcoholConsump', -0.09485313995926549),
 ('CholCheck', 0.11538161710270915),
 ('Stroke', 0.12542678468516733),
 ('PhysActivity', -0.15866560486405157),
 ('Education', -0.17048063498806143),
 ('HeartDiseaseorAttack', 0.21152340436022687),
 ('PhysHlth', 0.21308101903810317),
 ('Income', -0.2244487149638171),
 ('DiffWalk', 0.272646006159808),
 ('Age', 0.27873806628188813),
 ('HighChol', 0.28921280708865016),
 ('BMI', 0.29337274476103575),
 ('HighBP', 0.3815155489073117),
 ('GenHlth', 0.4076115984949182),
 ('Diabetes_binary', 1.0)]

In [28]:
keep_features = map(lambda b: b[0], filter(lambda a: abs(a[1]) > 0.25, correlation_map))

In [29]:
modified_dataset = raw_dataset[list(keep_features)]

In [30]:
logistic_regression_model(modified_dataset)

convergence after 38 epochs took 1 seconds
[[0.0702186  0.08500237 0.29923042 0.03782045 0.39012718 0.30067847]]
              precision    recall  f1-score   support

         0.0       0.75      0.73      0.74      7010
         1.0       0.74      0.76      0.75      7129

    accuracy                           0.74     14139
   macro avg       0.74      0.74      0.74     14139
weighted avg       0.74      0.74      0.74     14139



# Feature selection result: Logistic regression
By reducing the feature count using the correlation matrix and only keeping faetures that have more meaningful relationship with the target featurse, Logistic regression not only converged faster, it also kept its accuracy.

In [31]:
gradient_boost_classifier_model(modified_dataset)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3619           0.0245            6.90s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


         2           1.3400           0.0224            7.39s
         3           1.3200           0.0203            7.24s
         4           1.3008           0.0178            7.18s
         5           1.2841           0.0163            7.52s
         6           1.2685           0.0151            7.54s
         7           1.2545           0.0136            7.64s
         8           1.2417           0.0127            7.70s
         9           1.2303           0.0117            7.57s
        10           1.2191           0.0112            7.60s
        11           1.2090           0.0101            7.60s
        12           1.1982           0.0100            7.52s
        13           1.1890           0.0094            7.45s
        14           1.1793           0.0088            7.42s
        15           1.1707           0.0078            7.44s
        16           1.1641           0.0067            7.54s
        17           1.1571           0.0076            7.47s
        

# Feature selection result: GradientBoost
GradientBoost was also capable of keeping its performance after feature selection. It is worthy of noting that tuning hyperparameters had a mild effect on this model but it was negligible.

In [32]:
dnn_model(modified_dataset)

Epoch 1/100
509/509 - 2s - loss: 2.9741 - precision_1: 0.5214 - recall_1: 0.4159 - val_loss: 1.4772 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - 2s/epoch - 4ms/step
Epoch 2/100
509/509 - 1s - loss: 1.3527 - precision_1: 0.5436 - recall_1: 0.4692 - val_loss: 1.2555 - val_precision_1: 0.5331 - val_recall_1: 0.9868 - 1s/epoch - 2ms/step
Epoch 3/100
509/509 - 1s - loss: 1.1727 - precision_1: 0.5754 - recall_1: 0.6129 - val_loss: 1.0844 - val_precision_1: 0.7782 - val_recall_1: 0.5414 - 1s/epoch - 2ms/step
Epoch 4/100
509/509 - 1s - loss: 1.0471 - precision_1: 0.6267 - recall_1: 0.6636 - val_loss: 0.9666 - val_precision_1: 0.7491 - val_recall_1: 0.6950 - 1s/epoch - 2ms/step
Epoch 5/100
509/509 - 1s - loss: 0.9486 - precision_1: 0.6753 - recall_1: 0.6951 - val_loss: 0.8899 - val_precision_1: 0.6659 - val_recall_1: 0.8861 - 1s/epoch - 2ms/step
Epoch 6/100
509/509 - 1s - loss: 0.8885 - precision_1: 0.6906 - recall_1: 0.7119 - val_loss: 0.8377 - val_precision_1: 0.7251 - val_recal

# Feature selection result: DNN
DNN also proved to be consistant after feature selection. It even had a mild improvement(which is again, negligible).

# Feature selection overall result
So to conclude, we were able to predict our target feature with an acceptable accuracy even after selecting a subsest of our features. The following is the list of remaining features which proved to be decisive: DiffWalk, Age, HighChol, BMI, HighBP, GenHlth


In [33]:
# rename functions to snake_case

In [52]:
def get_feature_name(count):
  return ["f{c}".format(c=i) for i in range(count)]

In [57]:
component_count = 15

all_features = list(raw_dataset.keys())

all_features.remove("Diabetes_binary")

pca = PCA(n_components = component_count)

pca_columns = pca.fit_transform(standardized_dataset[all_features])

In [58]:
pca_dataset = pd.DataFrame()
pca_dataset["Diabetes_binary"] = standardized_dataset["Diabetes_binary"]
pca_dataset[get_feature_name(component_count)] =  pca_columns

In [59]:
logistic_regression_model(pca_dataset)

convergence after 34 epochs took 1 seconds
[[ 0.32456242 -0.22470491  0.03210159 -0.06513454  0.10273124  0.20779652
   0.02765655  0.01666475 -0.03813325  0.0327819   0.05444129 -0.01668668
   0.01500069  0.02064489  0.00038825]]
              precision    recall  f1-score   support

         0.0       0.74      0.73      0.74      7010
         1.0       0.74      0.76      0.75      7129

    accuracy                           0.74     14139
   macro avg       0.74      0.74      0.74     14139
weighted avg       0.74      0.74      0.74     14139



In [60]:
gradient_boost_classifier_model(pca_dataset)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3612           0.0251            1.22m
         2           1.3384           0.0226            1.29m
         3           1.3179           0.0210            1.28m
         4           1.2981           0.0182            1.28m
         5           1.2813           0.0174            1.18m
         6           1.2650           0.0155            1.12m
         7           1.2502           0.0142            1.07m
         8           1.2371           0.0133            1.03m
         9           1.2247           0.0123            1.00m
        10           1.2139           0.0115           58.44s
        11           1.2016           0.0104           57.19s
        12           1.1933           0.0100           56.75s
        13           1.1833           0.0092           55.64s
        14           1.1737           0.0083           54.65s
        15           1.1655           0.0075           53.76s
       

In [61]:
dnn_model(pca_dataset)

Epoch 1/100
509/509 - 4s - loss: 2.3829 - precision_2: 0.6952 - recall_2: 0.6226 - val_loss: 1.4191 - val_precision_2: 0.7522 - val_recall_2: 0.7150 - 4s/epoch - 7ms/step
Epoch 2/100
509/509 - 1s - loss: 1.3206 - precision_2: 0.7415 - recall_2: 0.6770 - val_loss: 1.2090 - val_precision_2: 0.7260 - val_recall_2: 0.7701 - 1s/epoch - 2ms/step
Epoch 3/100
509/509 - 1s - loss: 1.1298 - precision_2: 0.7393 - recall_2: 0.6891 - val_loss: 1.0408 - val_precision_2: 0.7337 - val_recall_2: 0.7594 - 1s/epoch - 3ms/step
Epoch 4/100
509/509 - 2s - loss: 1.0268 - precision_2: 0.7412 - recall_2: 0.7000 - val_loss: 1.0087 - val_precision_2: 0.7623 - val_recall_2: 0.6763 - 2s/epoch - 3ms/step
Epoch 5/100
509/509 - 2s - loss: 0.9632 - precision_2: 0.7365 - recall_2: 0.7061 - val_loss: 0.9069 - val_precision_2: 0.7198 - val_recall_2: 0.7927 - 2s/epoch - 3ms/step
Epoch 6/100
509/509 - 1s - loss: 0.9081 - precision_2: 0.7326 - recall_2: 0.7198 - val_loss: 0.8936 - val_precision_2: 0.7149 - val_recall_2: 0.8

# Feature reduction: PCA
Principle component analysis is a technique which is used to decrease feature count such that their information is preserved, only in lower dimensions. It mainly aims to keep the variance of the original data in fewer columns. Result however, did not show any significant improvement, suggesting that important relations between features and target are already captured.  



# Final result
for the purpose of our research, we will use the following weights obtained from the Logistic regression model to apply on users' input.

### feature selected model:
$$[0.0702186,  0.08500237, 0.29923042, 0.03782045, 0.39012718, 0.30067847]$$

### original model:
$$[ 0.73642276,  0.5859419,   1.35431797,  0.07424616, -0.00842411,  0.17406674
   0.23032686, -0.02977515, -0.03619828, -0.08796805, -0.72938801,  0.05957254,
   0.01305635,  0.58596997, -0.00528506, -0.00804519,  0.13615712,  0.26243553,
   0.15226778, -0.02630124, -0.0588881 ]$$